In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Salary_Data.csv to Salary_Data.csv


In [ ]:
import os
import shutil
shutil.move('Salary_Data.csv', '/content/Salary_Data.csv')

'/content/Salary_Data.csv'

In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("optimizeExample").getOrCreate()

In [ ]:
df=spark.read.csv("/content/Salary_Data.csv",header=True,inferSchema=True)

In [ ]:
df.show(5)

+---+------+---------------+-----------------+-------------------+------+
|Age|Gender|Education Level|        Job Title|Years of Experience|Salary|
+---+------+---------------+-----------------+-------------------+------+
| 32|  Male|     Bachelor's|Software Engineer|                5.0| 90000|
| 28|Female|       Master's|     Data Analyst|                3.0| 65000|
| 45|  Male|            PhD|   Senior Manager|               15.0|150000|
| 36|Female|     Bachelor's|  Sales Associate|                7.0| 60000|
| 52|  Male|       Master's|         Director|               20.0|200000|
+---+------+---------------+-----------------+-------------------+------+
only showing top 5 rows



In [ ]:
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Education Level: string (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Years of Experience: double (nullable = true)
 |-- Salary: integer (nullable = true)



In [ ]:
df.groupBy("Age").count().show()

+----+-----+
| Age|count|
+----+-----+
|  31|  365|
|  53|    7|
|  34|  309|
|  28|  429|
|  26|  394|
|  27|  517|
|  44|  126|
|  22|   15|
|  47|   47|
|NULL|    2|
|  52|   29|
|  40|   92|
|  57|    9|
|  54|   68|
|  48|   98|
|  41|  129|
|  43|  158|
|  37|  156|
|  61|    2|
|  35|  200|
+----+-----+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import avg

# Corrected code with unique aliases
df.groupBy("Salary").agg(
    avg("Years of Experience").alias("Average Years of Experience"),
    avg("Age").alias("Average Age")
).show()

+------+---------------------------+------------------+
|Salary|Average Years of Experience|       Average Age|
+------+---------------------------+------------------+
|135000|          9.534246575342467|35.821917808219176|
|182237|                       22.0|              49.0|
|113000|                       10.0|              37.0|
| 85000|          5.850467289719626|31.018691588785046|
|145000|          8.859649122807017| 34.05263157894737|
| 75969|                        4.0|              27.0|
| 70216|                        3.0|              26.0|
|106492|                        8.0|              36.0|
|184006|                       24.0|              48.0|
|113334|                       10.0|              33.0|
| 67556|                        3.0|              26.0|
|128078|                       11.0|              32.0|
|146000|                       14.0|              43.0|
|173582|                       21.0|              47.0|
|182392|                       23.0|            

In [ ]:

# Choose a valid column for analysis
valid_column = "saving"  # Change to any suitable column from your dataset

In [ ]:

# Import the required module
import time






In [ ]:
# Measure execution time with caching
df.cache()  # Cache the DataFrame
start_time_cached = time.time()
df.groupBy("Age").count().show()
end_time_cached = time.time()
print("Execution Time (Without Cache):", end_time_cached - start_time_cached)

+----+-----+
| Age|count|
+----+-----+
|  31|  365|
|  53|    7|
|  34|  309|
|  28|  429|
|  26|  394|
|  27|  517|
|  44|  126|
|  22|   15|
|  47|   47|
|NULL|    2|
|  52|   29|
|  40|   92|
|  57|    9|
|  54|   68|
|  48|   98|
|  41|  129|
|  43|  158|
|  37|  156|
|  61|    2|
|  35|  200|
+----+-----+
only showing top 20 rows

Execution Time (Without Cache): 0.8961763381958008


In [ ]:

# Cache the DataFrame
df.cache()

# Measure execution time after caching
start_time = time.time()
df.groupBy("Salary").count().show()
end_time = time.time()
print("Execution Time (With Cache):", end_time - start_time)




+------+-----+
|Salary|count|
+------+-----+
|135000|   73|
|182237|    1|
|113000|    8|
| 85000|  107|
|145000|   57|
| 75969|    1|
| 70216|    4|
|106492|    2|
|184006|    1|
|113334|    3|
| 67556|    1|
|128078|    2|
|146000|    8|
|173582|    1|
|182392|    3|
| 91000|    1|
|152944|    1|
|183020|    1|
|188288|    1|
|134979|    1|
+------+-----+
only showing top 20 rows

Execution Time (With Cache): 0.5915489196777344


In [ ]:
from pyspark import StorageLevel

df.persist(StorageLevel.MEMORY_AND_DISK)

start_time = time.time()
df.groupBy("Salary").count().show()
end_time = time.time()
print("Execution Time (With Persist):", end_time - start_time)

df.unpersist()


+------+-----+
|Salary|count|
+------+-----+
|135000|   73|
|182237|    1|
|113000|    8|
| 85000|  107|
|145000|   57|
| 75969|    1|
| 70216|    4|
|106492|    2|
|184006|    1|
|113334|    3|
| 67556|    1|
|128078|    2|
|146000|    8|
|173582|    1|
|182392|    3|
| 91000|    1|
|152944|    1|
|183020|    1|
|188288|    1|
|134979|    1|
+------+-----+
only showing top 20 rows

Execution Time (With Persist): 0.3278172016143799


DataFrame[Age: int, Gender: string, Education Level: string, Job Title: string, Years of Experience: double, Salary: int]

In [ ]:

df_repartitioned = df.repartition(10)

print("Number of partitions after repartitioning:", df_repartitioned.rdd.getNumPartitions())


df_coalesced = df.repartition(10).coalesce(5)

print("Number of partitions after coalescing:", df_coalesced.rdd.getNumPartitions())



Number of partitions after repartitioning: 10
Number of partitions after coalescing: 5


In [ ]:
df_selected = df.select("Salary", "Age", "Gender")
df_selected.show(5)

+------+---+------+
|Salary|Age|Gender|
+------+---+------+
| 90000| 32|  Male|
| 65000| 28|Female|
|150000| 45|  Male|
| 60000| 36|Female|
|200000| 52|  Male|
+------+---+------+
only showing top 5 rows



In [ ]:

df.write.parquet("/mnt/data/optimized.parquet")

In [ ]:
row_count = df.select("Age").distinct().count()
print("Number of unique users:", row_count)



Number of unique users: 42


In [ ]:

spark.conf.set("spark.sql.adaptive.enabled", "true")

In [ ]:

import time

start_time = time.time()
df_filtered = df.filter(df.Salary == "false").groupBy("Gender").count()
df_filtered.show()
end_time = time.time()
print("Execution Time (Without Cache):", end_time - start_time)

df.cache()
start_time = time.time()
df_filtered = df.filter(df.Salary == "false").groupBy("Gender").count()
df_filtered.show()
end_time = time.time()
print("Execution Time (With Cache):", end_time - start_time)


+------+-----+
|Gender|count|
+------+-----+
+------+-----+

Execution Time (Without Cache): 0.11007237434387207
+------+-----+
|Gender|count|
+------+-----+
+------+-----+

Execution Time (With Cache): 0.05798006057739258


In [ ]:



print("Number of partitions:", df.rdd.getNumPartitions())


Number of partitions: 1


In [ ]:

df_repartitioned = df.repartition(8)
print("Partitions after repartition:", df_repartitioned.rdd.getNumPartitions())



Partitions after repartition: 8


In [ ]:


df_coalesced = df.repartition(8).coalesce(4)
print("Partitions after coalesce:", df_coalesced.rdd.getNumPartitions())


Partitions after coalesce: 4


In [ ]:

df_unique = df.dropDuplicates(["Salary", "Age"])
df_unique.show()



+----+------+-----------------+--------------------+-------------------+------+
| Age|Gender|  Education Level|           Job Title|Years of Experience|Salary|
+----+------+-----------------+--------------------+-------------------+------+
|NULL|  NULL|             NULL|                NULL|               NULL|  NULL|
|  26|Female|Bachelor's Degree|            Social M|               NULL|  NULL|
|  31|  Male|  Master's Degree| Full Stack Engineer|                8.0|  NULL|
|  36|  Male|Bachelor's Degree|      Sales Director|                6.0|  NULL|
|  29|  Male|       Bachelor's|Junior Business O...|                1.5|   350|
|  31|Female|Bachelor's Degree|Junior HR Coordin...|                4.0|   500|
|  25|Female|Bachelor's Degree| Front end Developer|                1.0|   550|
|  23|  Male|              PhD|Software Engineer...|                1.0|   579|
|  21|Female|      High School|Junior Sales Repr...|                0.0| 25000|
|  22|  Male|      High School|Junior Sa

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def check_Salary(Salary):
    return "High" if Salary == "high" else "Low"

udf_check_Salary = udf(check_Salary, StringType())

df = df.withColumn("Years of Experience", udf_check_Salary(df.Salary))
df.show()


+---+------+---------------+--------------------+-------------------+------+
|Age|Gender|Education Level|           Job Title|Years of Experience|Salary|
+---+------+---------------+--------------------+-------------------+------+
| 32|  Male|     Bachelor's|   Software Engineer|                Low| 90000|
| 28|Female|       Master's|        Data Analyst|                Low| 65000|
| 45|  Male|            PhD|      Senior Manager|                Low|150000|
| 36|Female|     Bachelor's|     Sales Associate|                Low| 60000|
| 52|  Male|       Master's|            Director|                Low|200000|
| 29|  Male|     Bachelor's|   Marketing Analyst|                Low| 55000|
| 42|Female|       Master's|     Product Manager|                Low|120000|
| 31|  Male|     Bachelor's|       Sales Manager|                Low| 80000|
| 26|Female|     Bachelor's|Marketing Coordin...|                Low| 45000|
| 38|  Male|            PhD|    Senior Scientist|                Low|110000|

In [ ]:


from pyspark.sql.functions import when, col

df = df.withColumn(
    " Job Title",
    when(col("Salary") == "high", "High").otherwise("Low")
)
df.show()


+---+------+---------------+--------------------+-------------------+------+----------+
|Age|Gender|Education Level|           Job Title|Years of Experience|Salary| Job Title|
+---+------+---------------+--------------------+-------------------+------+----------+
| 32|  Male|     Bachelor's|   Software Engineer|                Low| 90000|       Low|
| 28|Female|       Master's|        Data Analyst|                Low| 65000|       Low|
| 45|  Male|            PhD|      Senior Manager|                Low|150000|       Low|
| 36|Female|     Bachelor's|     Sales Associate|                Low| 60000|       Low|
| 52|  Male|       Master's|            Director|                Low|200000|       Low|
| 29|  Male|     Bachelor's|   Marketing Analyst|                Low| 55000|       Low|
| 42|Female|       Master's|     Product Manager|                Low|120000|       Low|
| 31|  Male|     Bachelor's|       Sales Manager|                Low| 80000|       Low|
| 26|Female|     Bachelor's|Mark